In [71]:
!pip install nltk
!pip install pickle
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
#from wordcloud import WordCloud
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Lasso
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

# Load your data (replace with your actual data loading)
# Assuming your data is in a CSV file named 'data.csv'
new_columns=pd.read_csv('azureml://subscriptions/512a781e-d15a-4734-adca-96ec827531cb/resourcegroups/xwang3306-rg/workspaces/DVA_PROJECT/datastores/workspaceblobstore/paths/UI/2024-11-05_152711_UTC/neiss_10p_sample_new_columns.csv')
data_10P = pd.read_csv('../xwang3306/NEISS_10P_Sample.csv')
sematic=pd.read_csv('../xwang3306/10P_sematic.csv')

ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle


[nltk_data] Downloading package punkt to /home/azureuser/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#data_10p=data.sample(frac=0.1,random_state=42).reset_index(drop=True)
#data_10p.to_csv('NEISS_10P_Sample.csv')

In [72]:
data=data_10P.merge(new_columns,how='inner',on='CPSC_Case_Number').merge(sematic,how='inner',on='CPSC_Case_Number').reset_index(drop=True)
data.head(5)

,Unnamed: 0,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Hispanic,Body_Part,Diagnosis,...,Stratum,PSU,Weight,Year,Month,Day,activity_at_injury,injury_mechanism,object_involved,sematic_distance
0,0,221032332,2022-09-24,14,1,0,0,0.0,34,71,...,V,77,17.2223,2022,9,24,playing basketball,fell,basketball,-0.026630
1,1,181109464,2018-10-30,28,1,1,0,0.0,79,71,...,V,25,17.5136,2018,10,30,pick up crate,bent,crate,0.000000
2,2,210103105,2020-10-24,35,1,0,0,0.0,30,53,...,S,27,76.0369,2020,10,24,riding on mountain bike practicing,fell,mountain bike,-0.074183
3,3,161157997,2016-11-15,214,2,0,0,0.0,76,53,...,S,48,85.2143,2016,11,15,coming down stairs,fell,stairs,-0.144378
4,4,181107411,2018-10-21,4,1,0,0,0.0,92,72,...,C,20,4.9383,2018,10,21,playing with toy kitchen appliance,caught,toy kitchen appliance,0.015949


In [53]:
#data['sematic_distance']=sematic_distance

In [73]:
import numpy as np
data['#_prod']=np.nan
data.loc[((data['Product_1']>0)), '#_prod'] = 1
data.loc[((data['Product_1']>0) & (data['Product_2']>0)), '#_prod'] = 2
data.loc[((data['Product_1']>0) & (data['Product_2']>0) & (data['Product_3']>0)), '#_prod'] = 3

In [74]:
import numpy as np
data['Disposition_recode']=np.nan
data.loc[((data['Disposition']==1)), 'Disposition_recode'] = 0
data.loc[((data['Disposition']==2)), 'Disposition_recode'] = 1
data.loc[((data['Disposition']==4)), 'Disposition_recode'] = 2
data.loc[((data['Disposition']==5)), 'Disposition_recode'] = 3
data.loc[((data['Disposition']==8)), 'Disposition_recode'] = 4
data=data[data['Disposition_recode'].notna()]

data['Disposition_recode_2']=0
data.loc[((data['Disposition_recode']>0)), 'Disposition_recode_2'] = 1

In [75]:
data['Disposition_recode'].value_counts()
data['Disposition_recode_2'].value_counts()

0    308854
1     37458
Name: Disposition_recode_2, dtype: int64

In [76]:
bdpt_dict={}
bdpt_dict[0]='INTERNAL'
bdpt_dict[30]='SHOULDER'
bdpt_dict[31]='UPPERTRUNK'
bdpt_dict[32]='ELBOW'
bdpt_dict[33]='LOWERARM'
bdpt_dict[34]='WRIST'
bdpt_dict[35]='KNEE'
bdpt_dict[36]='LOWERLEG'
bdpt_dict[37]='ANKLE'
bdpt_dict[38]='PUBICREGION'
bdpt_dict[75]='HEAD'
bdpt_dict[76]='FACE'
bdpt_dict[77]='EYEBALL'
bdpt_dict[78]='UPPERTRUNK(OLD)'
bdpt_dict[79]='LOWERTRUNK'
bdpt_dict[80]='UPPERARM'
bdpt_dict[81]='UPPERLEG'
bdpt_dict[82]='HAND'
bdpt_dict[83]='FOOT'
bdpt_dict[84]='25-50% OF BODY'
bdpt_dict[85]='ALLPARTSBODY'
bdpt_dict[86]='OTHER(OLD)'
bdpt_dict[87]='NOTSTATED/UNK'
bdpt_dict[88]='MOUTH'
bdpt_dict[89]='NECK'
bdpt_dict[90]='LOWERARM(OLD)'
bdpt_dict[91]='LOWERLEG(OLD)'
bdpt_dict[92]='FINGER'
bdpt_dict[93]='TOE'
bdpt_dict[94]='EAR'


In [77]:
data['body_string']=data['Body_Part'].map(bdpt_dict)

In [78]:
data['Narrative_LLM']=data["activity_at_injury"].astype(str) + ' '+data["injury_mechanism"].astype(str)+ ' ' + data["object_involved"].astype(str)

In [45]:
import spacy
import requests

# Load the SpaCy model
nlp = spacy.load('en_core_web_md')

sematic_distance=[]

for i in range(len(data)):
    #print(i)
    word1 = data.at[i,'body_string'] #"cut"
    word2 = data.at[i,'Narrative_LLM'] #"bike"

    # SpaCy similarity
    doc1 = nlp(word1)
    doc2 = nlp(word2)
    similarity = doc1.similarity(doc2)
    sematic_distance.append(similarity)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

/tmp/ipykernel_20402/438897136.py:17: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarity = doc1.similarity(doc2)


In [79]:
def remove_after_dx(narrative):
  if isinstance(narrative, str):
    parts = narrative.split("DX", 1)
    if len(parts) > 1:
      return parts[0]
    else:
      return narrative  # No "DX:" found, return the original string
  else:
    return narrative  # Not a string, return as is


replace_list=['ANKLE', 'ARM', 'BODY_PART', 'CHEST', 'CONTUSION', 'CUT', 'EAR', 'ELBOW', 'EYE', 'FACE', 'FINGER', 'FOOT', 'FOREHEAD', 'FRACTURE', 'FX', 'HAND', 'HEAD', 'HIP', 'KNEE', 'LAC', 'LACERATION', 'LEG', 'LOC', 'LOSE', 'NECK', 'PAIN', 'SHOULDER', 
'SPRAIN', 'STRAIN', 'SWELL', 'THUMB', 'TOE', 'WRIST','ABRASION', 'ACHE', 'BREAK', 'BURN', 'CHIN', 'CUT', 'ER', 'FRACTURE', 'FX', 'HIT', 'INJURY', 'LACERATION', 'LIP', 'LOSE', 'LOC', 'MOUTH', 'NOSE', 'PAIN', 'RIB', 'SCALP', 'SPRAIN', 'STRAIN', 'SWELL', 'TOE', 'TWIST', 'WRIST']

for i in replace_list:
  data['Narrative'] = data['Narrative'].str.replace(i, '')
  data['Narrative_LLM']= data['Narrative_LLM'].str.replace(i, '')

data['Narrative'] = data['Narrative'].apply(remove_after_dx)
data['Narrative'] = data['Narrative'].str.replace('YOM', '')
data['Narrative'] = data['Narrative'].str.replace('YOF', '')
data['Narrative'] = data['Narrative'].str.replace('YR', '')
data['Narrative'] = data['Narrative'].str.replace('OLD', '')
data['Narrative'] = data['Narrative'].str.replace('MALE', '')
data['Narrative'] = data['Narrative'].str.replace('FEMALE', '')
data['Narrative'] = data['Narrative'].str.replace(' YO ', '')
data['Narrative'] = data['Narrative'].str.replace('YO ', '')
data['Narrative'] = data['Narrative'].str.replace(' F ', '')
data['Narrative'] = data['Narrative'].str.replace('YF', '')
data['Narrative'] = data['Narrative'].str.replace(' M ', '')
data['Narrative'] = data['Narrative'].str.replace('ACCIDENTALLY','')
data['Narrative'] = data['Narrative'].str.replace('AGO', '')
data['Narrative'] = data['Narrative'].str.replace('TODAY', '')
data['Narrative'] = data['Narrative'].str.replace('YESTERDAY', '')
data['Narrative'] = data['Narrative'].str.replace('PATIENT', '')
data['Narrative'] = data['Narrative'].str.replace(' PT ', '')
data['Narrative'] = data['Narrative'].str.replace('INJURY', '')
data['Narrative'] = data['Narrative'].str.replace('REPORT', '')
data['Narrative'] = data['Narrative'].str.replace('HURT', '')
data['Narrative'] = data['Narrative'].str.replace('INJ', '')
data['Narrative'] = data['Narrative'].str.replace('FELL', 'FALL')
data['Narrative'] = data['Narrative'].str.replace('INJURE', '')
data['Narrative'] = data['Narrative'].str.replace('JURED', '')
data['Narrative'] = data['Narrative'].str.replace('URED', '')
data['Narrative'] = data['Narrative'].str.replace(' ED', '')
data['Narrative'] = data['Narrative'].str.replace(' RT ', '')
data['Narrative'] = data['Narrative'].str.replace(' LT ', '')

data['Narrative_LLM'] = data['Narrative_LLM'].str.replace('fell', 'fall')


In [60]:
#data_out=data[['CPSC_Case_Number','sematic_distance']]
#data_out.to_csv('10P_sematic.csv',index=False)

In [80]:
core_col=['Disposition_recode_2','sematic_distance','Age', 'Sex', 'Location','Body_Part','Product_1' ,'Narrative','Narrative_LLM'] # 'Alcohol', 'Drug',
data_core=data[core_col]
#data_core['sematic_distance']=sematic_distance


In [81]:
data_core=data_core.sample(frac=1,random_state=42).reset_index(drop=True)
data_test=data_core.head(3463).reset_index(drop=True)
data_fit=data_core.tail(342849).reset_index(drop=True)

In [82]:
df_bad=data_fit[data_fit['Disposition_recode_2']==1]
df_good=data_fit[data_fit['Disposition_recode_2']==0]

In [83]:
data_good_sample=df_good.sample(frac=0.1,random_state=42).reset_index(drop=True)
data_bad_sample=df_bad.sample(frac=0.8,random_state=42).reset_index(drop=True)
data_core_sample=pd.concat([data_good_sample,data_bad_sample]).reset_index(drop=True)

In [84]:
data_core_sample=data_core_sample.sample(frac=1,random_state=42).reset_index(drop=True)
data_core_sample

,Disposition_recode_2,sematic_distance,Age,Sex,Location,Body_Part,Product_1,Narrative,Narrative_LLM
0,1,0.000000,2,1,9,85,1284,2 SWIMMING IN PUBLIC POOL AND TOOK OFF *** AND...,swimming submersion swimming pool
1,1,0.000000,210,2,0,85,1931,10 MONTHWITH *** INGESTION 800 MG ADMITTED,unknown ingestion 800 mg tablet or capsule drugs
2,0,-0.102001,26,1,4,92,5040,26 BICYCLIST WAS TRYING TO AVOID PEOPLE WHEN H...,trying to avoid people fall bike tire
3,0,0.000000,7,2,0,80,5040,7 FALL OFF A BIKE AND LANDED ON,riding bike fall bike
4,0,-0.055345,22,1,0,83,1819,22 STEPPED ON A NAIL AND IT WENT INTO RIGHT,stepped on a nail stepped on nail
...,...,...,...,...,...,...,...,...,...
60225,1,-0.100344,84,1,1,75,679,,unknown fall couch
60226,1,0.000000,66,1,1,31,1807,66 BRUISING TO WALL WHEN UNWITNESSED FALL TO ...,falling from bed fall bed
60227,0,-0.075132,63,2,1,83,611,63 SPED GETTING IN SHOW ON SHOW DOOR,getting in shower slipped shower door
60228,0,-0.101458,34,2,1,88,550,34 WAS WATG A VIDEO ON H PHONE WHEN THE FLASHE...,watching a video on her phone seizure phone


In [85]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')


corpus = data_core_sample['Narrative'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

data_core_sample['Processed_Narrative'] = corpus.apply(preprocess_text)

# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer(max_features=70, stop_words='english')

# Fit and transform the processed text
tfidf_matrix = vectorizer.fit_transform(data_core_sample['Processed_Narrative'])

with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

# Convert the TF-IDF matrix to a DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())



[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/azureuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [86]:
corpus_LLM = data_core_sample['Narrative_LLM'].fillna('')

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalpha() and token not in stopwords.words('english')]
    tagged_tokens = nltk.pos_tag(tokens)
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token, pos='v') for token, pos in tagged_tokens]
    return ' '.join(lemmatized_tokens)

data_core_sample['Processed_Narrative_LLM'] = corpus_LLM.apply(preprocess_text)

# Create a TfidfVectorizer object
vectorizer_LLM = TfidfVectorizer(max_features=70, stop_words='english')



# Fit and transform the processed text
tfidf_matrix_LLM = vectorizer_LLM.fit_transform(data_core_sample['Processed_Narrative_LLM'])

with open('tfidf_vectorizer_LLM.pkl', 'wb') as file:
    pickle.dump(vectorizer_LLM, file)

# Convert the TF-IDF matrix to a DataFrame
tfidf_df_LLM = pd.DataFrame(tfidf_matrix_LLM.toarray(), columns=vectorizer_LLM.get_feature_names_out())
tfidf_df_LLM=tfidf_df_LLM.add_suffix('_LLM')

# Concatenate the TF-IDF features with your existing data
data_ready = pd.concat([data_core_sample, tfidf_df,tfidf_df_LLM], axis=1)
#data_ready = pd.concat([data_core_sample, tfidf_df], axis=1)

In [87]:
new_data=data_test.copy()
corpus_2 = new_data['Narrative'].fillna('')
new_data['Processed_Narrative'] = corpus_2.apply(preprocess_text)
tfidf_new = vectorizer.transform(new_data['Processed_Narrative'])
tfidf_df_new = pd.DataFrame(tfidf_new.toarray(), columns=vectorizer.get_feature_names_out())

corpus_LLM_2 = new_data['Narrative_LLM'].fillna('')
new_data['Processed_Narrative_LLM'] = corpus_LLM_2.apply(preprocess_text)
tfidf_LLM_new = vectorizer_LLM.transform(new_data['Processed_Narrative_LLM'])
tfidf_df_new_LLM = pd.DataFrame(tfidf_LLM_new.toarray(), columns=vectorizer_LLM.get_feature_names_out())
tfidf_df_new_LLM=tfidf_df_new_LLM.add_suffix('_LLM')

data_ready_test = pd.concat([data_test, tfidf_df_new,tfidf_df_new_LLM], axis=1)
#data_ready_test = pd.concat([data_test, tfidf_df_new], axis=1)

In [17]:

#list(data_ready_test.columns)==list(data_ready_test.columns)

In [18]:
#data_ready = pd.concat([data_core_sample, tfidf_df], axis=1)
#list(data_ready.columns)

In [88]:
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_auc_score
from scipy.stats import mode
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

# Separate features (X) and target variable (y)
X = data_ready.drop(['Disposition_recode_2','Narrative','Narrative_LLM','Processed_Narrative','Processed_Narrative_LLM'], axis=1)  # ,'Processed_Narrative'] Replace 'target_variable' with your target column name
y = data_ready['Disposition_recode_2']

# Encode target variable if it's categorical
le = LabelEncoder()
y = le.fit_transform(y)

# Split data into training and testing sets
X_train, _, y_train, _ = train_test_split(X, y, test_size=0.2, random_state=42)

X_test=data_ready_test.drop(['Disposition_recode_2','Narrative','Narrative_LLM'], axis=1)
y_test=data_ready_test['Disposition_recode_2']
y_test = le.fit_transform(y_test)

# Create a scaler for numerical features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.select_dtypes(include=['number']))

with open('X_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

X_test_scaled = scaler.transform(X_test.select_dtypes(include=['number']))

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.select_dtypes(include=['number']).columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.select_dtypes(include=['number']).columns)

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled_df, y_train)

# Combine scaled numerical features with categorical features
X_train_final = X_resampled.copy()
X_test_final = X_test_scaled_df.copy()
y_train_final=y_resampled.copy()


# Define models and their parameter grids for hyperparameter tuning
models = {
    #'KNN': (KNeighborsClassifier(), {'knn__n_neighbors': [3, 5, 7]}),
    #'Random_Forest': (RandomForestClassifier(), {'random_forest__n_estimators': [100, 200], 'random_forest__max_depth': [4, 8]}),
    'XGBoost': (XGBClassifier(objective='binary:logistic', use_label_encoder=False), 
           {'xgboost__learning_rate': [0.1, 0.01, 0.001],            # Step size shrinkage to prevent overfitting
            'xgboost__max_depth': [3, 5, 7],                         # Maximum depth of each tree
            #'xgboost__n_estimators': [100, 200, 300],                # Number of boosting rounds
            #'xgboost__min_child_weight': [1, 3, 5],                  # Minimum sum of instance weight needed in a child
            #'xgboost__subsample': [0.6, 0.8,0.9],                   # Subsample ratio of the training instances
            #'xgboost__colsample_bytree': [0.6, 0.8, 0.9],            # Subsample ratio of columns when constructing each tree
            #'xgboost__gamma': [0.1, 0.5, 1,3],                     # Minimum loss reduction required for a split
            #'xgboost__reg_alpha': [0.01, 0.1,0.5,1],                    # L1 regularization term on weights
            #'xgboost__reg_lambda': [1, 1.5, 2],                      # L2 regularization term on weights
            'xgboost__scale_pos_weight': [1, 5, 10]
            }),
    #'Logistic_Regression': (LogisticRegression(solver='liblinear'), {'logistic_regression__C': [0.1, 1, 10]}),
    #'SVM': (SVC(probability=True), {'svm__C': [0.1, 1, 10], 'svm__kernel': ['linear', 'rbf']}),
    #'NN': (MLPClassifier(max_iter=1000), {'nn__hidden_layer_sizes': [(10,), (50,), (100,)], 'nn__activation': ['relu', 'tanh']}),
}

# Initialize list to store predictions from each model
ensemble_predictions = []
ensemble_probabilities = []

for model_name, (model, param_grid) in models.items():
    print(f"Training {model_name}...")

    # Create a pipeline for preprocessing and model training
    pipeline = Pipeline([
        (model_name.lower(), model)
    ])

    # Perform hyperparameter tuning using GridSearchCV with AUC score
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, scoring='precision_weighted',n_jobs=-1)
    grid_search.fit(X_train_final, y_train_final)

    # Get the best model from the grid search
    best_model = grid_search.best_estimator_

    # Make predictions and predict probabilities on the test set
    y_pred = best_model.predict(X_test_final)
    y_prob = best_model.predict_proba(X_test_final)[:, 1]  # Probability for the positive class
    ensemble_predictions.append(y_pred)
    ensemble_probabilities.append(y_prob)

    # Evaluate the model using accuracy, F1-score, and AUC
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    print(f"{model_name} Accuracy: {accuracy}")
    print(f"{model_name} F1 Score: {f1}")
    print(f"{model_name} AUC: {auc}")
    print(classification_report(y_test, y_pred))

# Implement ensemble method (majority voting for predictions, average for probabilities)
ensemble_predictions = np.array(ensemble_predictions)
ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()

# Average the probabilities for the ensemble AUC
ensemble_probabilities = np.mean(ensemble_probabilities, axis=0)

# Evaluate the ensemble model using accuracy, F1-score, and AUC
ensemble_accuracy = accuracy_score(y_test, ensemble_pred_final)
ensemble_f1 = f1_score(y_test, ensemble_pred_final)
ensemble_auc = roc_auc_score(y_test, ensemble_probabilities)
print("Ensemble Accuracy:", ensemble_accuracy)
print("Ensemble F1 Score:", ensemble_f1)
print("Ensemble AUC:", ensemble_auc)
print(classification_report(y_test, ensemble_pred_final))

with open('xgboost_tfidf_LLM_v2.pkl', 'wb') as file:
    pickle.dump(best_model, file)

Training XGBoost...
[05:13:40] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBoost Accuracy: 0.7692751949177015
XGBoost F1 Score: 0.43851018973998596
XGBoost AUC: 0.8529394700416906
              precision    recall  f1-score   support

           0       0.97      0.77      0.85      3070
           1       0.30      0.79      0.44       393

    accuracy                           0.77      3463
   macro avg       0.63      0.78      0.65      3463
weighted avg       0.89      0.77      0.81      3463

Ensemble Accuracy: 0.7692751949177015
Ensemble F1 Score: 0.43851018973998596
Ensemble AUC: 0.8529394700416906
              precision    recall  f1-score   support

           0       0.97      0.77      0.85      3070
           1       0.30      0.79      0.44       393

    accuracy        

/tmp/ipykernel_20402/2236807770.py:102: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  ensemble_pred_final = mode(ensemble_predictions, axis=0)[0].flatten()


In [20]:
xxx

NameError: name 'xxx' is not defined

In [ ]:
pip install contextualSpellCheck
pip install spacy

In [ ]:
import spacy
import contextualSpellCheck

nlp = spacy.load('en_core_web_sm')
contextualSpellCheck.add_to_pipe(nlp)
doc = nlp('Income was $9.4 milion compared to the prior year of $2.7 milion.')

In [ ]:
print(dir(pickle))

In [ ]:
import pickle
pickle.__version__

In [ ]:
pip freeze